In [1]:
#導入函數庫
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

#read All dataframe df from ANN_training_data.CSV
file_name='ANN_training_data_AC_combination.CSV'
filepath='C:\\Users\\richard.weng\\Documents\\Python Scripts\\python_projects\\(1) NIVG Project\\ANN\\'
file_data = filepath+file_name
df=pd.read_csv(file_data)

#只取特定排做dataframe
#使用iloc的方式，若只寫一個column，會變成只有取值，不是變成dataframe
columnNum = 8 # 要取的column數，也就是輸入變數
testNum = 12 # 要取的測試組數，也就是測試的次數
dataFrame_x = df.iloc[0:testNum,0:columnNum]
print(dataFrame_x)
dataFrame_y = df.iloc[0:testNum,0:1]
print(dataFrame_y)

#只取特定排做dataframe 2
#columnNum = 8
#column = []
#for i in range (columnNum):
    #column.append(i+1)
#dataFrame_x = pd.read_csv(file_data,usecols=column)
#dataFrame_y = pd.read_csv(file_data,usecols=[0])

#轉成array
row_data_x = np.array(dataFrame_x)
row_data_y = np.array(dataFrame_y)
print(np.shape(row_data_x))
print(np.shape(row_data_y))

# 假設70%訓練，30%要驗證 (TrainingData and TestingData)
x_train, x_test, y_train, y_test = train_test_split(row_data_x,row_data_y,test_size=0.3,random_state=50)
print(np.shape(x_train))
print(np.shape(y_train))

# scale units
X = x_train/np.amax(x_train, axis=0) # maximum of x_train array
xPredicted = x_test/np.amax(x_test, axis=0) # maximum of xPredicted (our input data for the prediction)
y = y_train/600 # max test score is 600
print(X)
print('------------------')
print(y)

    Glucose     940nm     970nm    1200nm    1300nm   940nm.1   970nm.1  \
0       119  0.010188  0.004748  0.001878  0.001826  0.319015  0.115179   
1       145  0.008460  0.004270  0.001739  0.001120  0.305339  0.121104   
2       204  0.010536  0.005383  0.002264  0.001350  0.257109  0.117797   
3       220  0.013245  0.006663  0.002355  0.001454  0.353507  0.153750   
4       249  0.029836  0.010786  0.002845  0.001771  0.199929  0.080159   
5       186  0.024736  0.011827  0.002829  0.003028  0.168685  0.076317   
6       266  0.032736  0.015472  0.003732  0.003028  0.240754  0.108069   
7       160  0.012435  0.004659  0.002332  0.001963  0.351204  0.122169   
8       218  0.017663  0.008322  0.003475  0.001831  0.411324  0.162264   
9       131  0.025846  0.001106  0.000490  0.000810  0.540108  0.067599   
10      189  0.028267  0.008843  0.003790  0.001825  0.480469  0.148393   
11      139  0.013587  0.005530  0.002364  0.001698  0.395931  0.146885   

    1200nm.1  
0   0.097

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):
    # add one more layer and return the output of this layer
    layer_name = 'layer%s' % n_layer
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
            tf.summary.histogram(layer_name + '/weights', Weights)
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
            tf.summary.histogram(layer_name + '/biases', biases)#RW類似正向傳播
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        # here to dropout
        Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
        tf.summary.histogram(layer_name + '/outputs', outputs)
    return outputs

# define placeholder for inputs to network
#Tensorflow 如果想要从外部传入data, 那就需要用到 tf.placeholder(), 然后以这种形式传输数据 sess.run(*, feed_dict={input: }).
#利用占位符定义我们所需的神经网络的输入。 tf.placeholder()就是代表占位符，
#这里的None代表无论输入有多少都可以，因为输入只有一个特征，所以这里是1

# define placeholder for inputs to network
with tf.name_scope('inputs'):
    keep_prob = tf.placeholder(tf.float32)
    xs = tf.placeholder(tf.float32, [None, 8], name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

# add hidden layer
l1 = add_layer(xs, 8, 4, n_layer=1, activation_function=tf.nn.tanh)
# add output layer
prediction = add_layer(l1, 4, 1, n_layer=2, activation_function=tf.nn.softmax)

# the error between prediciton and real data
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))
    tf.summary.scalar('loss', cross_entropy)

with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# important step
sess = tf.Session()
merged = tf.summary.merge_all()

# summary writer goes in here
train_writer = tf.summary.FileWriter("logs/train", sess.graph)
test_writer = tf.summary.FileWriter("logs/test", sess.graph)

# tf.initialize_all_variables() no long valid from
# 2017-03-02 if using tensorflow >= 0.12
init = tf.global_variables_initializer()
sess.run(init)

# # plot the real data
# fig = plt.figure()
# ax = fig.add_subplot(1,1,1)
# ax.scatter(x_data, y_data)
# plt.ion()
# plt.show()

#训练
for i in range(500):
    # here to determine the keeping probability
    sess.run(train_step, feed_dict={xs: x_train, ys: y_train, keep_prob: 1})
    if i % 50 == 0:
        # record loss
        print(sess.run(cross_entropy, feed_dict={xs: x_train, ys: y_train}))
        train_result = sess.run(merged, feed_dict={xs: x_train, ys: y_train, keep_prob: 1})
        test_result = sess.run(merged, feed_dict={xs: x_test, ys: y_test, keep_prob: 1})
        train_writer.add_summary(train_result, i)
        test_writer.add_summary(test_result, i)

# plot the prediction
plt.scatter(y_data, prediction_value)
plt.title('ANN Simulation Result')
plt.xlabel('Input Y')
plt.ylabel('Predicted Y')
plt.grid()
plt.show()
# direct to the local dir and run this in terminal:
# $ tensorboard --logdir logs

InvalidArgumentError: You must feed a value for placeholder tensor 'inputs_2/Placeholder' with dtype float
	 [[node inputs_2/Placeholder (defined at <ipython-input-4-9b909500c734>:33)  = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'inputs_2/Placeholder', defined at:
  File "C:\Users\richard.weng\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\richard.weng\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\richard.weng\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\richard.weng\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C

In [5]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

def add_layer(inputs, in_size, out_size, n_layer, activation_function=None):
    # add one more layer and return the output of this layer
    layer_name = 'layer%s' % n_layer
    with tf.name_scope(layer_name):
        with tf.name_scope('weights'):
            Weights = tf.Variable(tf.random_normal([in_size, out_size]), name='W')
        with tf.name_scope('biases'):
            biases = tf.Variable(tf.zeros([1, out_size]) + 0.1, name='b')
        with tf.name_scope('Wx_plus_b'):
            Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
        # here to dropout
        Wx_plus_b = tf.nn.dropout(Wx_plus_b, keep_prob)
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b, )
    return outputs

# define placeholder for inputs to network
#Tensorflow 如果想要从外部传入data, 那就需要用到 tf.placeholder(), 然后以这种形式传输数据 sess.run(*, feed_dict={input: }).
#利用占位符定义我们所需的神经网络的输入。 tf.placeholder()就是代表占位符，
#这里的None代表无论输入有多少都可以，因为输入只有一个特征，所以这里是1

# define placeholder for inputs to network
with tf.name_scope('inputs'):
    keep_prob = tf.placeholder(tf.float32)
    xs = tf.placeholder(tf.float32, [None, 8], name='x_input')
    ys = tf.placeholder(tf.float32, [None, 1], name='y_input')

# add hidden layer
l1 = add_layer(xs, 8, 4, n_layer=1, activation_function=tf.nn.tanh)
# add output layer
prediction = add_layer(l1, 4, 1, n_layer=2, activation_function=tf.nn.softmax)

# the error between prediciton and real data
with tf.name_scope('loss'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(ys * tf.log(prediction),
                                              reduction_indices=[1]))
with tf.name_scope('train'):
    train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# important step
sess = tf.Session()
merged = tf.summary.merge_all()

# tf.initialize_all_variables() no long valid from
init = tf.global_variables_initializer()
sess.run(init)

# # plot the real data
# fig = plt.figure()
# ax = fig.add_subplot(1,1,1)
# ax.scatter(x_data, y_data)
# plt.ion()
# plt.show()

#训练
for i in range(500):
    # here to determine the keeping probability
    sess.run(train_step, feed_dict={xs: x_train, ys: y_train, keep_prob: 1})
    if i % 50 == 0:
        # record loss
        print(sess.run(cross_entropy, feed_dict={xs: x_train, ys: y_train}))

# plot the prediction
plt.scatter(y_data, prediction_value)
plt.title('ANN Simulation Result')
plt.xlabel('Input Y')
plt.ylabel('Predicted Y')
plt.grid()
plt.show()

InvalidArgumentError: You must feed a value for placeholder tensor 'inputs_3/Placeholder' with dtype float
	 [[node inputs_3/Placeholder (defined at <ipython-input-5-1350dcbbf515>:30)  = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'inputs_3/Placeholder', defined at:
  File "C:\Users\richard.weng\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\richard.weng\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\richard.weng\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\richard.weng\AppData\Local\Continuum\anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C